Suppose that 50 people are given a placebo and
50 are given a new treatment.
30 placebo patients show improvement while
40 treated patients show improvement.
Let $\tau = p_1 - p_2$ where
$p_2$ is the probability of improving under treatment and
$p_1$ is the probability of improving under placebo.

In [7]:
from collections import namedtuple
import numpy as np
import scipy.stats

(a) Find the MLE of $\tau$.
Find the standard error and
90 percent confidence interval using the delta method.

In [8]:
"""

Since we very often pass the tuple (n1, s1, n2, s2)
as an argument, we give that tuple a name 'data'.

n1: number of patients in treatment 1
s1: number of successes in treatment 1
n2: number of patients in treatment 2
s2: number of successes in treatment 2

"""

data = namedtuple('data', ['n1', 's1', 'n2', 's2'])

def tau_mle(data):
    
    return data.s2/data.n2 - data.s1/data.n1

def std_err_mle_tau(data):
    
    p1_mle = data.s1/data.n1
    p2_mle = data.s2/data.n2
    
    return np.sqrt(
        p1_mle*(1-p1_mle)/data.n1 + p2_mle*(1-p2_mle)/data.n2
    )

def normal_interval(estimate, std_error, alpha):
    
    z = scipy.stats.norm.isf(alpha/2)
    
    lower_bound = estimate - z*std_error
    upper_bound = estimate + z*std_error
    
    return lower_bound, upper_bound

def interval_mle_tau(data, alpha):
    
    return normal_interval(
        estimate=tau_mle(current_data),
        std_error=std_err_mle_tau(current_data),
        alpha=alpha
    )

def interval_text(interval):
    """ Expects interval = (lower_bound, upper_bound). """
    
    lower_bound, upper_bound = interval
    
    return f"({lower_bound:.4f}, {upper_bound:.4f})"

In [3]:
# Report the results

current_data = data(50, 30, 50, 40)
alpha = 0.1

print(
    "-----------\n"
    "METHOD: MLE\n"
    "-----------\n"
    f"The MLE for tau is {tau_mle(current_data):.3f}.\n"
    f"The standard error estimate is {std_err_mle_tau(current_data):.4f}.\n"
    f"The 90 percent confidence interval is {
        interval_text(interval_mle_tau(current_data, alpha))
    }."
)

-----------
METHOD: MLE
-----------
The MLE for tau is 0.200.
The standard error estimate is 0.0894.
The 90 percent confidence interval is (0.0529, 0.3471).


(b) Find the standard error and 90 percent confidence interval using the parametric bootstrap.

In [1]:
def boot_samples(data, B):
    
    p1_mle = data.s1/data.n1
    p2_mle = data.s2/data.n2
    
    bernoulli1_boot_replications = scipy.stats.bernoulli.rvs(p=p1_mle, size=[data.n1, B])
    bernoulli2_boot_replications = scipy.stats.bernoulli.rvs(p=p2_mle, size=[data.n2, B])
    
    return bernoulli1_boot_replications, bernoulli2_boot_replications

def tau_mle_boot_replications(data, B):

    bernoulli1_boot_replications, bernoulli2_boot_replications = boot_samples(data, B)
    
    boot_sample_mean1 = bernoulli1_boot_replications.mean(axis=0)
    boot_sample_mean2 = bernoulli2_boot_replications.mean(axis=0)
    
    return boot_sample_mean2 - boot_sample_mean1

def std_err_boot(data, B):
    
    return tau_mle_boot_replications(data, B).std()

def interval_boot_normal(data, alpha, B):

    return normal_interval(
        estimate=tau_mle(current_data),
        std_error=std_err_boot(current_data, B=B),
        alpha=alpha
    )

def percentile_interval(replications, alpha):
    
    lower_bound = np.quantile(replications, q=alpha/2)
    upper_bound = np.quantile(replications, q=1 - alpha/2)
    
    return lower_bound, upper_bound

def interval_boot_percentile(data, alpha, B):
    
    return percentile_interval(
        replications=tau_mle_boot_replications(data, B),
        alpha=alpha
    )

In [6]:
# Report the results

current_data = data(50, 30, 50, 40)
alpha = 0.1
B = int(1e5)

print(
    "-----------------\n"
    "METHOD: Bootstrap\n"
    "-----------------\n"
    f"A standard error estimate is {std_err_boot(current_data, B):.4f}.\n"
    f"A 90 percent confidence Normal interval is {
        interval_text(interval_boot_normal(current_data, alpha, B))
    }.\n"
    f"A 90 percent confidence percentile interval is {
        interval_text(interval_boot_percentile(current_data, alpha, B))
    }.\n\n"
    "Note: the standard error and the confidence intervals\n"
    "come from **different** bootstrap replication runs."
)

-----------------
METHOD: Bootstrap
-----------------
A standard error estimate is 0.0893.
A 90 percent confidence Normal interval is (0.0529, 0.3471).
A 90 percent confidence percentile interval is (0.0600, 0.3400).

Note: the standard error and the confidence intervals
come from **different** bootstrap replication runs.


(c) Use the prior $f(p_1, p_2) = 1$.
Use simulation to find the posterior mean and posterior 90 percent interval for $\tau$.

In [12]:
def sim_samples(data, B):
    
    bernoulli1_sim_replications = scipy.stats.beta.rvs(a=data.s1, b=data.n1-data.s1, size=B)
    bernoulli2_sim_replications = scipy.stats.beta.rvs(a=data.s2, b=data.n2-data.s2, size=B)
    
    return bernoulli1_sim_replications, bernoulli2_sim_replications

def tau_mle_sim_replications(data, B):
    
    bernoulli1_sim_replications, bernoulli2_sim_replications = sim_samples(data, B)
    
    return bernoulli2_sim_replications - bernoulli1_sim_replications

def post_mean_tau_sim(data, B):
    
    return tau_mle_sim_replications(data, B).mean()

def std_err_tau_sim(data, B):
    
    return tau_mle_sim_replications(data, B).std()

def interval_tau_sim_normal(data, alpha, B):

    return normal_interval(
        estimate=post_mean_tau_sim(data=data, B=B),
        std_error=std_err_tau_sim(data=data, B=B),
        alpha=alpha
    )
    
def interval_tau_sim_percentile(data, alpha, B):
    
    return percentile_interval(
        replications=tau_mle_sim_replications(data, B),
        alpha=alpha
    )

In [18]:
# Report the results

current_data = data(50, 30, 50, 40)
alpha = 0.1
B = int(1e6)

print(
    "----------------------------\n"
    "METHOD: Posterior simulation\n"
    "----------------------------\n"
    f"A posterior mean estimate for tau is {post_mean_tau_sim(current_data, B):.3f}.\n"
    f"A standard error estimate is {std_err_tau_sim(current_data, B):.3f}.\n"
    f"A 90 percent confidence interval is {
        interval_text(interval_tau_sim_normal(current_data, alpha, B))
    }.\n"
    f"A 90 percent confidence percentile interval is {
        interval_text(interval_tau_sim_percentile(current_data, alpha, B))
    }.\n\n"
    "Note: the standard error and the confidence intervals\n"
    "come from **different** simulation runs."
)

----------------------------
METHOD: Posterior simulation
----------------------------
A posterior mean estimate for tau is 0.200.
A standard error estimate is 0.088.
A 90 percent confidence interval is (0.0543, 0.3459).
A 90 percent confidence percentile interval is (0.0530, 0.3442).

Note: the standard error and the confidence intervals
come from **different** simulation runs.


(d) Let
$$ \psi = \log \left( \frac{p_1}{1-p_1} \div \frac{p_2}{1-p_2} \right)$$
be the log-odds ratio.
Note that $\psi = 0$ if $p_1 = p_2$.
Find the MLE of $\psi$.
Use the delta method to find a 90 percent confidence interval for $\psi$.

In [15]:
def log_odds(p1, p2):
    
    return np.log(
        ( p1/(1-p1) ) / ( p2/(1-p2) )
    )

def psi_mle(data):
    
    p1_mle = data.s1/data.n1
    p2_mle = data.s2/data.n2
    
    return log_odds(p1_mle, p2_mle)
    
def std_err_mle_psi(data):
    
    p1_mle = data.s1/data.n1
    p2_mle = data.s2/data.n2
    
    return np.sqrt(
        1/( data.n1 * p1_mle * (1 - p1_mle) )
        + 1/( data.n2 * p2_mle * (1 - p2_mle) )
    )
    
def interval_mle_psi(data, alpha):
    
    return normal_interval(
        estimate=psi_mle(current_data),
        std_error=std_err_mle_psi(current_data),
        alpha=alpha
    )

In [10]:
# Report the results

current_data = data(50, 30, 50, 40)
alpha = 0.1

print(
    "-----------\n"
    "METHOD: MLE\n"
    "-----------\n"
    f"The MLE for psi is {psi_mle(current_data):.3f}.\n"
    f"The standard error estimate is {std_err_mle_psi(current_data):.4f}.\n"
    f"The 90 percent confidence interval is {
        interval_text(interval_mle_psi(current_data, alpha))
    }."
)

-----------
METHOD: MLE
-----------
The MLE for psi is -0.981.
The standard error estimate is 0.4564.
The 90 percent confidence interval is (-1.7316, -0.2301).


(e) Use simulation to find the posterior mean and posterior 90 percent interval for $\psi$.

In [16]:
def psi_mle_sim_replications(data, B):
    
    bernoulli1_sim_replications, bernoulli2_sim_replications = sim_samples(data, B)
    
    return log_odds(bernoulli1_sim_replications, bernoulli2_sim_replications)

def post_mean_psi_sim(data, B):
    
    return psi_mle_sim_replications(data, B).mean()

def std_err_psi_sim(data, B):
    
    return psi_mle_sim_replications(data, B).std()

def interval_psi_sim_normal(data, alpha, B):

    return normal_interval(
        estimate=post_mean_psi_sim(data=data, B=B),
        std_error=std_err_psi_sim(data=data, B=B),
        alpha=alpha
    )
    
def interval_psi_sim_percentile(data, alpha, B):
    
    return percentile_interval(
        replications=psi_mle_sim_replications(data, B),
        alpha=alpha
    )

In [17]:
# Report the results

current_data = data(50, 30, 50, 40)
alpha = 0.1
B = int(1e6)

print(
    "----------------------------\n"
    "METHOD: Posterior simulation\n"
    "----------------------------\n"
    f"A posterior mean estimate for psi is {post_mean_psi_sim(current_data, B):.3f}.\n"
    f"A standard error estimate is {std_err_psi_sim(current_data, B):.3f}.\n"
    f"A 90 percent confidence interval is {
        interval_text(interval_psi_sim_normal(current_data, alpha, B))
    }.\n"
    f"A 90 percent confidence percentile interval is {
        interval_text(interval_psi_sim_percentile(current_data, alpha, B))
    }.\n\n"
    "Note: the standard error and the confidence intervals\n"
    "come from **different** simulation runs."
)

----------------------------
METHOD: Posterior simulation
----------------------------
A posterior mean estimate for psi is -1.010.
A standard error estimate is 0.464.
A 90 percent confidence interval is (-1.7749, -0.2470).
A 90 percent confidence percentile interval is (-1.7852, -0.2575).

Note: the standard error and the confidence intervals
come from **different** simulation runs.


### Important observation
Note that we do **not** expect the posterior mean of $\psi$
to approach the "true" value
$$h(0.6, 0.8) \approx -0.98$$
where $h$ denotes the log-odds function such that $\psi = h(p_1,p_2)$.
Indeed, this posterior mean is not *supposed* to approximate this value.
Instead, this posterior mean is an approximation of the integral
$$
\int\int h(p_1, p_1) f_\text{Beta}(p_1;50,30) f_\text{Beta}(p_2;50,40) dp_1 dp_2.
$$
So what we are seeing is that this integral is $\approx -1.01$.

Note that, as the sample size grows larger
(provided the proportions of improved patients stay the same in both groups)
this product of Beta distributions
will approach a Dirac mass centered at $(p_1, p_2) = (0.6, 0.8)$ such that
$$
\lim_{n\to\infty} \int\int h(p_1, p_1) f_\text{Beta}(p_1;5n,3n) f_\text{Beta}(p_2;5n,4n) dp_1 dp_2
= h(0.6, 0.8) \approx -0.98.
$$